In [22]:
import os
from dotenv import load_dotenv
load_dotenv()
from metaphor_python import Metaphor
import openai
from IPython.display import Markdown, display

openai.api_key = os.getenv("OPENAI_API_KEY")
metaphor = Metaphor(os.getenv("METAPHOR_API_KEY"))

In [162]:
def get_llm_response(system='You are a helpful assistant.', user = '', temperature = 1, model = 'gpt-3.5-turbo'):
    completion = openai.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[
            {'role': 'system', 'content': system},
            {'role': 'user', 'content': user},
        ]
    )
    return completion.choices[0].message.content

In [175]:
PROPERTY_TOPIC = 'Vintage Clark Kent figurine, possibly from Kenner 1986 "Super Powers Collection."'
STRUCTURE_TOPIC = 'Purchase roof shingles'

In [165]:
def create_keyword_query_generation_prompt(topic, n):
    return f"""I'm writing a research report on {topic} and need help coming up with Google keyword search queries.
Google keyword searches should just be a few words long. It should not be a complete sentence.
Please generate a diverse list of {n} Google keyword search queries that would be useful for writing a research report on ${topic}. Do not add any formatting or numbering to the queries."""

print(get_llm_response(
    system='The user will ask you to help generate some search queries. Respond with only the suggested queries in plain text with no extra formatting, each on it\'s own line.',
    user=create_keyword_query_generation_prompt(PROPERTY_TOPIC, 3),
))


vintage Clark Kent figurine
Kenner Super Powers Collection figurine
1986 Clark Kent figurine Kenner


In [206]:
def generate_search_queries(topic, n):
    user_prompt = create_keyword_query_generation_prompt(topic, n)
    completion = get_llm_response(
        system='The user will ask you to help generate some search queries. Respond with only the suggested queries in plain text with no extra formatting, each on it\'s own line.',
        user=user_prompt,
        temperature=1
    )
    queries = [s for s in completion.split('\n') if s.strip()][:n]
    return queries

In [176]:
prop_queries = generate_search_queries(PROPERTY_TOPIC, 3)
struct_queries = generate_search_queries(STRUCTURE_TOPIC, 3)

In [177]:
print(prop_queries)
print(struct_queries)

['vintage Clark Kent figurine', 'Kenner Super Powers Collection Clark Kent', '1986 Clark Kent action figure']
['purchase roof shingles', 'types of roof shingles', 'best roof shingles']


In [178]:
def get_search_results(queries, type, linksPerQuery=1):
    results = []
    for query in queries:
        search_response = metaphor.search(query, type=type, num_results=linksPerQuery, use_autoprompt=False)
        results.extend(search_response.results)
    return results

In [179]:
struct_links = get_search_results(struct_queries, 'keyword')

In [245]:
def display_base_models(base_models):
    """
    Display the title, url, and published_date of each BaseModel object in a list.

    Args:
        base_models (List[BaseModel]): List of BaseModel objects.

    """
    for model in base_models:
        print(f"Title: {model.title}")
        print(f"URL: {model.url}")
        print(f"Published Date: {model.published_date}")

In [246]:
display_base_models(struct_links)

Title: Roof Shingles - The Home Depot
URL: https://www.homedepot.com/b/Building-Materials-Roofing-Roof-Shingles/N-5yc1vZc5rb
Published Date: None
Title: What Are The Different Types Of Roof Shingles? (2023) - Architectural Digest
URL: https://www.architecturaldigest.com/reviews/roofing/types-of-roof-shingles
Published Date: 2023-07-24T12:00:00.000Z
Title: Best Roofing Shingles of 2023 - Consumer Reports
URL: https://www.consumerreports.org/home-garden/roofing/best-roofing-shingles-from-consumer-reports-tests-a3871500341/
Published Date: 2023-11-18T12:00:00.000Z


In [244]:
for link in struct_links:
    print(link)

Title: Roof Shingles - The Home Depot
URL: https://www.homedepot.com/b/Building-Materials-Roofing-Roof-Shingles/N-5yc1vZc5rb
ID: 9117d1df-7c2f-47ed-aced-1530dcb382b7
Score: None
Published Date: None
Author: None
Extract: None
Title: What Are The Different Types Of Roof Shingles? (2023) - Architectural Digest
URL: https://www.architecturaldigest.com/reviews/roofing/types-of-roof-shingles
ID: 1bd4e0cb-7dba-4c44-a8bc-d522a7b1ecdf
Score: None
Published Date: 2023-07-24T12:00:00.000Z
Author: None
Extract: None
Title: Best Roofing Shingles of 2023 - Consumer Reports
URL: https://www.consumerreports.org/home-garden/roofing/best-roofing-shingles-from-consumer-reports-tests-a3871500341/
ID: 0ab2dbb9-5113-4cfb-9a14-8e3e9ff3baf8
Score: None
Published Date: 2023-11-18T12:00:00.000Z
Author: None
Extract: None


In [181]:
prop_links = get_search_results(prop_queries, 'keyword')

In [243]:
for link in prop_links:
    print(link)

In [183]:
def get_page_contents(search_results):
    contents_response = metaphor.get_contents(search_results)
    return contents_response.contents

In [184]:
struct_content = get_page_contents([link.id for link in struct_links])

In [190]:
Markdown(f"{struct_content[1].extract}")

<div><div>
<div>
<p>
</p><div>
<ul>
<li>
<a href="#types-of-roof-shingles">
	Types of Roof Shingles	</a>
</li>
<li>
<a href="#three-tab-asphalt-shingles">
	Three-Tab Asphalt Shingles	</a>
</li>
<li>
<a href="#-architectural-asphalt-shingles">
	 Architectural Asphalt Shingles	</a>
</li>
<li>
<a href="#clay-tiles">
	Clay Tiles	</a>
</li>
<li>
<a href="#concrete-tiles">
	Concrete Tiles	</a>
</li>
<li>
<a href="#copper-shingles">
	Copper Shingles	</a>
</li>
<li>
<a href="#plastic-shingles">
	Plastic Shingles	</a>
</li>
<li>
<a href="#slate-tiles">
	Slate Tiles	</a>
</li>
<li>
<a href="#steel-shingles">
	Steel Shingles	</a>
</li>
<li>
<a href="#solar-shingles">
	Solar Shingles	</a>
</li>
<li>
<a href="#wood-shakes">
	Wood Shakes	</a>
</li>
<li>
<a href="#pro-vs.-diy">
	Pro vs. DIY	</a>
</li>
<li>
<a href="#our-recommendation">
	Our Recommendation	</a>
</li>
<li>
<a href="#types-of-roof-shingles-faq">
	FAQ	</a>
</li>
</ul>
</div>
</div>
<figure></figure>
<p><strong>Affiliate Disclaimer:</strong> All products and services featured are independently selected by our editors. However, when you buy something through our retail links, we may earn an affiliate commission.</p>
<p><a href="https://www.architecturaldigest.com/reviews/roofing/shingle-roof-cost"></a></p>
<p></p>
<hr/>
<h2>Types of Roof Shingles</h2>
<p></p>
<figure><table><thead><tr><th>Shingle Type</th><th>Life Span*</th></tr></thead>
<tbody><tr>
<td>
<p>Three-tab asphalt shingles</p>
</td>
<td>
<p>15–20 years</p>
</td>
</tr>
<tr>
<td>
<p>Architectural asphalt shingles</p>
</td>
<td>
<p>20–30 years</p>
</td>
</tr>
<tr>
<td>
<p>Clay tiles</p>
</td>
<td>
<p>50–100 years</p>
</td>
</tr>
<tr>
<td>
<p>Concrete tiles</p>
</td>
<td>
<p>40–80 years</p>
</td>
</tr>
<tr>
<td>
<p>Copper shingles</p>
</td>
<td>
<p>70+ years</p>
</td>
</tr>
<tr>
<td>
<p>Plastic shingles</p>
</td>
<td>
<p>20–50 years</p>
</td>
</tr>
<tr>
<td>
<p>Slate tiles</p>
</td>
<td>
<p>50–100 years</p>
</td>
</tr>
<tr>
<td>
<p>Steel shingles</p>
</td>
<td>
<p>40–70 years</p>
</td>
</tr>
<tr>
<td>
<p>Solar shingles</p>
</td>
<td>
<p>20–25 years</p>
</td>
</tr>
<tr>
<td>
<p>Wood shakes</p>
</td>
<td>
<p>15–40 years</p>
</td>
</tr>
</tbody></table></figure>
<p><em></em></p>
<hr/>
<h2>Three-Tab Asphalt Shingles</h2>
<p></p>
<p></p>
<h3>Benefits and Drawbacks of Three-Tab Asphalt Shingles</h3>
<figure><table><thead><tr><th>Benefits</th><th>Drawbacks</th></tr></thead>
<tbody><tr>
<td>
<p>Low-cost</p>
</td>
<td>
<p>Not the best choice for climates with frequent, extreme weather events</p>
</td>
</tr>
<tr>
<td>
<p>Available in many colors</p>
</td>
<td>
<p>Not environmentally friendly</p>
</td>
</tr>
<tr>
<td>
<p>Complements many architectural styles</p>
</td>
<td>
<p>Requires more frequent maintenance and replacement than other types of roof shingles</p>
</td>
</tr>
</tbody></table></figure>
<hr/>
<h2> Architectural Asphalt Shingles</h2>
<p></p>
<p></p>
<h3>Benefits and Drawbacks of Architectural Asphalt Shingles</h3>
<figure><table><thead><tr><th>Benefits</th><th>Drawbacks</th></tr></thead>
<tbody><tr>
<td>
<p>More sophisticated appearance than three-tab shingles</p>
</td>
<td>
<p>May break down in extreme climates</p>
</td>
</tr>
<tr>
<td>
<p>Straightforward installation and repairs</p>
</td>
<td>
<p>Only specially rated shingles are suitable for high-wind areas</p>
</td>
</tr>
<tr>
<td>
<p>Available in various colors and styles, some with protective finishes</p>
</td>
<td>
<p>Require maintenance and regular inspections to achieve maximum life span</p>
</td>
</tr>
</tbody></table></figure>
<hr/>
<h2>Clay Tiles</h2>
<p></p>
<p><a href="https://www.architecturaldigest.com/reviews/roofing/tile-roof-cost"></a></p>
<h3>Benefits and Drawbacks of Clay Tiles</h3>
<figure><table><thead><tr><th>Benefits</th><th>Drawbacks</th></tr></thead>
<tbody><tr>
<td>
<p>Attractive natural appearance</p>
</td>
<td>
<p>Need extra structural support due to heavy weight</p>
</td>
</tr>
<tr>
<td>
<p>Environmentally friendly</p>
</td>
<td>
<p>Installation requires a highly skilled roofing contractor</p>
</td>
</tr>
<tr>
<td>
<p>High durability and resistant to rot, insects, and fire</p>
</td>
<td>
<p>One of the most costly types of roof shingles available</p>
</td>
</tr>
</tbody></table></figure>
<hr/>
<h2>Concrete Tiles</h2>
<p></p>
<p></p>
<h3>Benefits and Drawbacks of Concrete Tiles</h3>
<figure><table><thead><tr><th>Benefits</th><th>Drawbacks</th></tr></thead>
<tbody><tr>
<td>
<p>Long life span of at least 40 years</p>
</td>
<td>
<p>Requires an experienced professional installer</p>
</td>
</tr>
<tr>
<td>
<p>Attractive appearance similar to more expensive natural materials</p>
</td>
<td>
<p>Very heavy material that requires substantial structural support</p>
</td>
</tr>
<tr>
<td>
<p>Requires virtually no maintenance</p>
</td>
<td>
<p>Not as environmentally friendly as clay or slate <a href="https://www.architecturaldigest.com/reviews/roofing/best-roof-shingles">roof shingles</a></p>
</td>
</tr>
</tbody></table></figure>
<hr/>
<h2>Copper Shingles</h2>
<p><a href="https://www.copper.org/applications/architecture/arch_dhb/arch-details/roofing/"></a></p>
<p></p>
<h3>Benefits and Drawbacks of Copper Shingles</h3>
<figure><table><thead><tr><th>Benefits</th><th>Drawbacks</th></tr></thead>
<tbody><tr>
<td>
<p>Has timeless beauty and develops an attractive patina</p>
</td>
<td>
<p>Is the most expensive metal roofing material</p>
</td>
</tr>
<tr>
<td>
<p>Requires minimal maintenance</p>
</td>
<td>
<p>Only available in natural copper color</p>
</td>
</tr>
<tr>
<td>
<p>Matches most architectural styles</p>
</td>
<td>
<p>Requires special care for proper installation</p>
</td>
</tr>
</tbody></table></figure>
<hr/>
<h2>Plastic Shingles</h2>
<p></p>
<p></p>
<h3>Benefits and Drawbacks of Plastic Shingles</h3>
<figure><table><thead><tr><th>Benefits</th><th>Drawbacks</th></tr></thead>
<tbody><tr>
<td>
<p>Impact- and rot-resistant roofing material</p>
</td>
<td>
<p>Limited availability in certain areas</p>
</td>
</tr>
<tr>
<td>
<p>Doesn’t require regular maintenance</p>
</td>
<td>
<p>Doesn’t break down over time, which can be environmentally damaging</p>
</td>
</tr>
<tr>
<td>
<p>Lightweight and doesn’t need extra structural support</p>
</td>
<td>
<p>Can be challenging to find a roofing contractor familiar with this installation</p>
</td>
</tr>
</tbody></table></figure>
<hr/>
<h2>Slate Tiles</h2>
<p></p>
<p><a href="https://www.architecturaldigest.com/reviews/roofing/roof-truss-cost"></a></p>
<h3>Benefits and Drawbacks of Slate Tiles</h3>
<figure><table><thead><tr><th>Benefits</th><th>Drawbacks</th></tr></thead>
<tbody><tr>
<td>
<p>Provides an elegant appearance that boosts curb appeal</p>
</td>
<td>
<p>Is incredibly heavy and requires extra support</p>
</td>
</tr>
<tr>
<td>
<p>Is durable and resists insects, rot, and extreme weather</p>
</td>
<td>
<p>Has limited color choices</p>
</td>
</tr>
<tr>
<td>
<p>Requires little to no maintenance</p>
</td>
<td>
<p>Is among the most costly roofing options available</p>
</td>
</tr>
</tbody></table></figure>
<hr/>
<h2>Steel Shingles</h2>
<p></p>
<p></p>
<h3>Benefits and Drawbacks of Steel Shingles</h3>
<figure><table><thead><tr><th>Benefits</th><th>Drawbacks</th></tr></thead>
<tbody><tr>
<td>
<p>Attractive designs that complement many home styles</p>
</td>
<td>
<p>More complicated than some other materials to repair if damage occurs</p>
</td>
</tr>
<tr>
<td>
<p>Reflective finishes can help reduce summer cooling costs</p>
</td>
<td>
<p>Amplifies the sound of rain if not insulated</p>
</td>
</tr>
<tr>
<td>
<p>Incredible durability and long life span</p>
</td>
<td>
<p>May get damaged by large hail</p>
</td>
</tr>
</tbody></table></figure>
<hr/>
<h2>Solar Shingles</h2>
<p></p>
<h3>Benefits and Drawbacks of Solar Shingles</h3>
<figure><table><thead><tr><th>Benefits</th><th>Drawbacks</th></tr></thead>
<tbody><tr>
<td>
<p>Produce green energy to supplement grid-based electricity</p>
</td>
<td>
<p>Shorter life span than many other roofing types, particularly at this price point</p>
</td>
</tr>
<tr>
<td>
<p>More attractive than bulky solar panels</p>
</td>
<td>
<p>Some homeowners associations do not allow solar shingles on homes</p>
</td>
</tr>
<tr>
<td>
<p>Can resist fire and withstand high winds</p>
</td>
<td>
<p>Technology may become outdated in a short period</p>
</td>
</tr>
</tbody></table></figure>
<hr/>
<h2>Wood Shakes</h2>
<p></p>
<p></p>
<h3>Benefits and Drawbacks of Wood Shakes</h3>
<figure><table><thead><tr><th>Benefits</th><th>Drawbacks</th></tr></thead>
<tbody><tr>
<td>
<p>Attractive rustic appearance</p>
</td>
<td>
<p>Require regular maintenance</p>
</td>
</tr>
<tr>
<td>
<p>Eco-friendly</p>
</td>
<td>
<p>Don’t last as long as other high-end roofing materials</p>
</td>
</tr>
<tr>
<td>
<p>Resistant to high winds and impacts</p>
</td>
<td>
<p>Typically unsuitable for homes in wildfire-prone areas</p>
</td>
</tr>
</tbody></table></figure>
<hr/>
<h2>Professional vs. DIY Roof Installation</h2>
<p><a href="https://www.architecturaldigest.com/reviews/roofing/new-roof-cost"></a></p>
<p><a href="https://www.architecturaldigest.com/reviews/roofing/soffit-replacement-cost"></a></p>
<p></p>
<ul>
<li></li>
<li></li>
<li></li>
<li></li>
<li></li>
</ul>
<hr/>
<h2>Our Recommendation</h2>
<p></p>
<p></p>
<hr/>
<div>
<h2>Types of Roof Shingles FAQ</h2>
<div>
<h3>What is reroofing?</h3>
<p>Reroofing is when a roofer installs new <a href="https://www.architecturaldigest.com/reviews/roofing/best-roof-shingles">roof shingles</a> over existing ones. This eliminates the need to remove old shingles and drastically lowers the roof repair cost. However, reroofing is not an option for heavier materials or homes that already have more than one layer of <a href="https://www.architecturaldigest.com/reviews/roofing/best-roof-shingles">roof shingles</a>.</p>
</div>
<div>
<h3>Should I buy extra shingles when I replace my roof?</h3>
<p>A skilled roofer will order the appropriate number of shingles to replace your roof. However, you may want to keep extra shingles for future repairs to ensure a match for color and texture.</p>
</div>
<div>
<h3>What is a roofing square?</h3>
<p>One roofing square equals 100 square feet. This measurement is commonly used to determine the number of roofing shingles needed for a project. Contractors often provide estimates with measurements based on the number of squares in a roof, so it’s important to familiarize yourself with this term.</p>
</div>
</div>
</div></div>

In [253]:
def synthesize_report(topic, search_contents, content_slice = 1000):
    inputData = ''.join([
        f'--START ITEM--\nURL: {item.url}\nCONTENT: {item.extract[:content_slice]}\n--END ITEM--\n'
        for item in search_contents
    ])
    return inputData
    # return get_llm_response(
    #     system='You are a helpful research assistant. Write a report according to the user\'s instructions.',
    #     user='Input Data:\n' + inputData + f'Write a two paragraph research report about {topic} based on the provided information. Include as many sources as possible. Provide citations in the text using footnote notation ([#]). First provide the report, followed by a single "References" section that lists all the URLs used, in the format [#] .',
    #     model='gpt-4' # want a better report? use gpt-4
    # )

In [254]:
prop_report = synthesize_report(STRUCTURE_TOPIC, struct_content)

In [255]:
print(f"{prop_report}")

--START ITEM--
URL: https://www.homedepot.com/b/Building-Materials-Roofing-Roof-Shingles/N-5yc1vZc5rb
CONTENT: <div><div><section><div><p></p><a href="https://www.homedepot.com/p/GAF-Timberline-Natural-Shadow-Charcoal-Algae-Resistant-Architectural-Shingles-33-33-sq-ft-per-Bundle-21-Pieces-0601180/100658149"><p></p></a><div><div><p></p><p>Buy or more </p></div><a href="https://www.homedepot.com/p/GAF-Timberline-Natural-Shadow-Charcoal-Algae-Resistant-Architectural-Shingles-33-33-sq-ft-per-Bundle-21-Pieces-0601180/100658149#ratings-and-reviews"></a></div></div><div><p></p><a href="https://www.homedepot.com/p/GAF-Timberline-Natural-Shadow-Charcoal-Algae-Resistant-Architectural-Shingles-33-33-sq-ft-per-Bundle-21-Pieces-0603180/328040780"><p></p></a><div><p></p><p>Buy or more </p></div></div><div><a href="https://www.homedepot.com/p/GAF-Timberline-HDZ-Charcoal-Algae-Resistant-Laminated-High-Definition-Shingles-33-33-sq-ft-per-Bundle-21-Pieces-0489180/309755006"><p></p></a><div><div><p></p><

In [234]:
def researcher(topic, n_queries, n_links_per_query):
    search_queries = generate_search_queries(topic, n_queries)
    print(search_queries)
    search_results = get_search_results(search_queries, 'keyword', n_links_per_query)
    search_contents = get_page_contents([link.id for link in search_results])
    report = synthesize_report(topic, search_contents)
    return report

In [235]:
research = researcher(PROPERTY_TOPIC, 2, 2)

['vintage Clark Kent figurine', 'Kenner 1986 Super Powers Collection']


In [236]:
print(f"{research}")

The Vintage Clark Kent action figure is potentially part of the 'Super Powers Collection,' a line of action figures produced and made available by Kenner Products, an American toy company, between the years 1984-1986[^2^]. The collection was known to feature characters from DC Comics[^2^], indicating a strong possibility that characters such as Clark Kent, the civilian identity of Superman, would have been part of the product line. With the figures compounded of high quality plastic material[^2^], the popularity and collectibility of these figures have not diminished over the years. 

On online platforms such as eBay, various iconic action figure collectibles including the Clark Kent figure can be found[^1^]. Fans and collectors alike have taken to such platforms to buy, sell or trade these items. Therefore, it is fair to conclude that even after decades of their initial release, these Kenner action figures, inclusive of the Clark Kent model, continue to sustain appeal in today's marke

In [241]:
response = metaphor.search("ceiling drywall",
    num_results=5,
    type='keyword',
    start_published_date="2023-06-12"
)

In [242]:
print(response)

Title: How to Hang Drywall Like a Pro (DIY) - Family Handyman
URL: https://www.familyhandyman.com/project/how-to-hang-drywall-like-a-pro/
ID: cfbd3af3-52c9-4457-b897-30805375d47f
Score: None
Published Date: 2023-09-26T12:00:00.000Z
Author: None
Extract: None

Title: USG Sheetrock Brand 1/2 in. x 4 ft. x 8 ft. UltraLight Drywall 14113411708
URL: https://www.homedepot.com/p/USG-Sheetrock-Brand-1-2-in-x-4-ft-x-8-ft-UltraLight-Drywall-14113411708/202530243
ID: 03c5ea62-5489-449d-bfef-d954951799d4
Score: None
Published Date: 2023-11-21T12:00:00.000Z
Author: None
Extract: None

Title: Sheetrock vs. Drywall: What's the Difference? - Angie's List
URL: https://www.angi.com/articles/sheetrock-vs-drywall.htm
ID: ae79c4e8-94fc-4d88-954b-39fab16a13ce
Score: None
Published Date: 2023-10-11T12:00:00.000Z
Author: None
Extract: None

Title: Drywall Ceiling Cost to Install or Replace (2023 Update) - HomeGuide
URL: https://homeguide.com/costs/drywall-ceiling-cost
ID: 346338ca-b367-468e-9daa-08a45dacd97c
